# Analyzing Arthropoda Taxonomy: Integrating GBIF, NCBI and iNaturalist Data

![title](https://wallpapercave.com/wp/wp1870417.jpg)

This Python notebook is designed for the purpose of integrating taxonomic data from two major biological databases, GBIF (Global Biodiversity Information Facility) and NCBI (National Center for Biotechnology Information) and iNaturalist, to enhance the accuracy and comprehensiveness of ecological and biological research. GBIF primarily focuses on biodiversity data including species distribution and ecological information, whereas NCBI provides a broader range of data including genomic and taxonomic details. 

Combining these sources enables researchers to cross-validate species identifications and improve the richness of ecological datasets with genetic information. A key biological task performed in this notebook is the construction of a taxonomic tree, which helps in visualizing and understanding the evolutionary relationships and classification hierarchy among different species within a chosen taxon (in this case, the Arthropda pyhlum).

## 1. Importing libraries

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import taxonmatch as txm

## 2. Downloading and processing samples

The initial steps involve downloading the most recent taxonomic data from GBIF and NCBI to ensure the analysis is based on the latest available information. 

In [3]:
gbif_dataset = txm.download_gbif_taxonomy()

GBIF backbone taxonomy data already downloaded.
Processing samples...
Done.


In [4]:
ncbi_dataset = txm.download_ncbi_taxonomy()

NCBI taxonomy data already downloaded.
Processing samples...
Done.


## 2.1 Checking Inconsistencies in nomenclature

Matching based on canonical names between the GBIF and NCBI datasets is unreliable due to significant taxonomic inconsistencies. In particular, the same canonical name may be assigned to multiple kingdoms, highlighting classification discrepancies. Even when the taxonomic status is accepted, the taxonomic structures in GBIF and NCBI can differ substantially. This necessitates filtering and evaluating differences before considering a match valid, preventing false correspondences between incongruent taxonomies.

In [5]:
df_inconsistencies = txm.get_inconsistencies(gbif_dataset, ncbi_dataset)

In [6]:
df_inconsistencies.sample(5)

,canonicalName,gbif_id,ncbi_id,gbif_rank,ncbi_rank,gbif_taxonomy,ncbi_taxonomy
566695,Salix alba,7882712,75704,species,species,chordata;ascidiacea;aplousobranchia;polycitoridae;salix;salix alba,streptophyta;magnoliopsida;malpighiales;salicaceae;salix;salix alba
205618,Bosea psychrotolerans,11618487,1871628,species,species,tracheophyta;magnoliopsida;caryophyllales;amaranthaceae;bosea;bosea psychrotolerans,pseudomonadota;alphaproteobacteria;hyphomicrobiales;boseaceae;bosea;bosea psychrotolerans
589774,Diploderma polygonatum,10561030,70539,species,species,basidiomycota;agaricomycetes;boletales;diplocystidiaceae;diploderma;diploderma polygonatum,chordata;lepidosauria;squamata;agamidae;diploderma;diploderma polygonatum
126987,Cyrtostrombidium paralongisomum,11200089,1537145,species,species,cyrtostrombidium;cyrtostrombidium paralongisomum,ciliophora;spirotrichea;cyrtostrombidiidae;cyrtostrombidium;cyrtostrombidium paralongisomum
125063,Euplotes daidaleos,8284379,471435,species,species,euplotes;euplotes daidaleos,ciliophora;spirotrichea;euplotida;euplotidae;euplotes;euplotes daidaleos


## 3.a Training the classifier model

If required, the notebook outlines steps to train a machine learning classifier to distinguish between correct and incorrect taxonomic matches. This involves generating positive and negative examples, preparing the training dataset, and comparing different models.

In [90]:
#positive_matches = txm.generate_positive_set(gbif_dataset, ncbi_dataset, 25000)

In [91]:
#negative_matches = txm.generate_negative_set(gbif_dataset, ncbi_dataset, 25000)

In [92]:
#full_training_set = txm.prepare_data(positive_matches, negative_matches)

In [93]:
#full_training_set.to_csv("training_set.txt", index = False)

In [94]:
#X_train, X_test, y_train, y_test = txm.generate_training_test(full_training_set)

In [95]:
#txm.compare_models(X_train, X_test, y_train, y_test)

In [96]:
#model = txm.XGBClassifier(learning_rate=0.1,n_estimators=500, max_depth=9, n_jobs=-1, colsample_bytree = 1, subsample = 0.8)

In [97]:
#model.fit(X_train, y_train, verbose=False)

In [98]:
#txm.save_model(model, "xgb_model")

## 3.b Load a pre-trained model

 Alternatively, it provides the option to load a pre-trained model, simplifying the process for routine analyses.

In [16]:
model = txm.load_xgb_model() 

## 4. Match NCBI with GBIF dataset 

In this section, the focus is on comparing and aligning the taxonomic data from NCBI and GBIF datasets. It specifically targets the taxon "Arthropoda" to narrow down the analysis to a specific family of bees. Using a pre-trained machine learning model, the notebook matches records from both datasets, categorizing them as exact matches, unmatched, or potentially mislabeled due to typographical errors

In [17]:
gbif_arthropoda, ncbi_arthropoda = txm.select_taxonomic_clade("arthropoda", gbif_dataset, ncbi_dataset)

In [18]:
matched_df, unmatched_df, possible_typos_df = txm.match_dataset(gbif_arthropoda, ncbi_arthropoda, model, tree_generation = True)

/Users/mleone1/miniconda3/envs/test_2/lib/python3.12/site-packages/taxonmatch/matching.py:276: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([matched_df, missing_ncbi_rows], ignore_index=True)


## 5. Generate the taxonomic tree 

In the last section, the notebook constructs a taxonomic tree from the matched and unmatched data between the GBIF and NCBI datasets, focusing on the Apidae family. This visual representation helps to illustrate the evolutionary relationships and classification hierarchy among the species. The tree is then converted into a dataframe for further analysis and saved in textual format for documentation and review purposes.

In [ ]:
tree = txm.generate_taxonomic_tree(matched_df, unmatched_df)

In [ ]:
#txm.print_tree(tree)

In [ ]:
cicadetta_tree = txm.reroot_tree(tree, root_name="cicadetta")

In [ ]:
txm.print_tree(cicadetta_tree)

In [ ]:
df_from_tree = txm.convert_tree_to_dataframe(tree, gbif_dataset[1], ncbi_dataset[1], "df_arthropoda.txt", index=False)

In [ ]:
txm.save_tree(tree, "./arthropoda_tree.txt", output_format='txt')

## 6. Add iNaturalist information

In [ ]:
inat_dataset = txm.download_inat_taxonomy()

In [ ]:
inat_tree = txm.add_inat_taxonomy(tree, inat_dataset)

In [ ]:
cicadetta_tree_with_inat = txm.reroot_tree(inat_tree, root_name="cicadetta")

In [ ]:
txm.print_tree(cicadetta_tree_with_inat)

In [ ]:
df_with_inat = txm.convert_tree_to_dataframe(inat_tree, gbif_dataset[1], ncbi_dataset[1], "df_arthropoda_with_inat.txt", inat_dataset=inat_dataset, index=True)

In [58]:
txm.save_tree(inat_tree, "arthropoda_tree_with_inat.txt", output_format='txt')

The tree is saved as TXT in the file: test_tree_final.txt.
